In [ ]:
suppressMessages(library(SeuratDisk))
suppressMessages(library(SingleR))
suppressMessages(library(Seurat))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(UCell))

In [ ]:
dataset <- readRDS("../../int_data/dataset_qc.rds")

In [ ]:
data <- SingleCellExperiment(assays = list(counts = dataset@assays@data[["soupX_counts"]]),
                             colData = dataset@colData)
rownames(data) = rownames(dataset)

In [ ]:
data <- as.Seurat(data, data = NULL)

In [ ]:
data

In [ ]:
data <- NormalizeData(data)
data <- ScaleData(data, features = row.names(data))
data <- FindVariableFeatures(data)
data <- RunPCA(data, verbose = F)
data <- FindNeighbors(data, verbose = F)
data <- FindClusters(data, verbose = F)
data <- RunUMAP(data, dims = 1:20, verbose = F)

In [ ]:
DimPlot(data, group.by = "Cell_type")

In [ ]:
DimPlot(data, label = T)
DimPlot(data, split.by = "Condition")
DimPlot(data, split.by = "batch")

In [ ]:
mouse_ref_immgen <- celldex::ImmGenData()

cell_classification <- SingleR(test = as.SingleCellExperiment(data),
                               ref = mouse_ref_immgen,
                               labels = mouse_ref_immgen$label.main)

data[["SingleR_label"]] <- cell_classification$labels
write.csv(data@meta.data, "dataset_metadata.csv")
plotScoreHeatmap(cell_classification)

In [ ]:
DimPlot(data, group.by = "SingleR_label", label = T, repel = T)

In [ ]:
data_neus <- data[, data$SingleR_label == "Neutrophils"]

In [ ]:
data_neus

In [ ]:
data_neus <- NormalizeData(data_neus)
data_neus <- ScaleData(data_neus, features = row.names(data_neus))
data_neus <- FindVariableFeatures(data_neus)
data_neus <- RunPCA(data_neus, verbose = F)
data_neus <- FindNeighbors(data_neus, verbose = F)
data_neus <- FindClusters(data_neus, verbose = F)
data_neus <- RunUMAP(data_neus, dims = 1:20, verbose = F)

In [ ]:
DimPlot(data_neus, group.by = "Condition")

In [ ]:
DimPlot(data_neus, group.by = "batch")

In [ ]:
DimPlot(data_neus, group.by = "Cell_type")

In [ ]:
write.csv(data@meta.data, "dataset_metadata.csv")